In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os

In [ ]:
from PIL import Image

In [ ]:
train_path='train/'
valid_path='valid/'

In [ ]:
os.listdir()

In [ ]:
train_img_path=pd.read_csv('train_image_paths.csv')
valid_img_path=pd.read_csv('valid_image_paths.csv')
train_labels=pd.read_csv('train_labeled_studies.csv')
valid_labels=pd.read_csv('valid_labeled_studies.csv')

In [ ]:
train_img_path.values[0]

In [ ]:
train_labels.values[0]

In [ ]:
train_img_path.shape,train_labels.shape

In [ ]:
train_img_path.values[:5]

In [ ]:
train_img_path['Label']=train_img_path.Img_Path.apply(lambda x:1 if 'positive' in x else 0)

In [ ]:
valid_img_path.shape,valid_labels.shape

In [ ]:
valid_img_path['Label']=valid_img_path.Img_Path.apply(lambda x:1 if 'positive' in x else 0)

In [ ]:
import torch
import torch.functional as F
from torch.utils.data import DataLoader,Dataset
from torchvision.transforms import Normalize,CenterCrop,Resize,Compose

In [ ]:
class Mura(Dataset):
    
    def __init__(self,df,root,transform=None):
        self.df=df
        self.root=root
        self.transform=transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        img_name=self.df.iloc[idx,0]
        img=Image.open(img_name)
        label=self.df.iloc[idx,1]
        if transform:
            img=self.transform(img)
        return img,label

In [ ]:
train_mura_dataset=Mura(df=train_img_path,root='../')

In [ ]:
from torchvision import transforms

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
train_mura_dataset=Mura(df=train_img_path,root='./',transform=data_transforms['train'])

In [ ]:
val_mura_dataset=Mura(df=valid_img_path,root='./',transform=data_transforms['val'])

In [ ]:
train_loader=DataLoader(dataset=train_mura_dataset,batch_size=64,num_workers=4,shuffle=True)

In [ ]:
val_loader=DataLoader(dataset=val_mura_dataset,batch_size=64,num_workers=4,shuffle=True)

In [ ]:
dataloaders={
    'train':train_loader,
    'val':val_loader
}

In [ ]:
dataset_sizes={
    'train':len(train_mura_dataset),
    'val':len(val_mura_dataset)
}

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

In [ ]:
torch.cuda.is_available()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model,criterion,optimizer,scheduler,num_epochs=1):
    start=time.time()
    
    best_model_wt=copy.deepcopy(model.state_dict())
    best_acc=0.0
    
    for epoch in range(num_epochs):
        print ('Epoch: {}/{}'.format(epoch+1,num_epochs))
        print ('-'*10)
        
        for phase in ['train','val']:
            if phase=='train':
                scheduler.step()
                model.train()
            else:
                model.eval()
            
            running_loss=0.0
            running_corrects=0
            
            for imgs,labels in dataloaders[phase]:
                imgs=imgs.to(device)
                labels=labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase=='train'):
                    outputs=model(imgs)
                    _,preds=torch.max(outputs)
                    loss=criterion(outputs,labels)
                    
                    if phase=='train':
                        loss.backward()
                        optimizer.step()
                
                running_loss+=loss.item()*imgs.size[0]
                running_corrects+=torch.sum(preds==labels)
                
                
            epoch_loss=running_loss/dataset_size[phase]
            epoch_acc=running_corrects.double()/dataset_size[phase]
            
            print ('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wt = copy.deepcopy(model.state_dict())
            
            print ()
            
        end=time.time()
        
        print('Training complete in {:.0f}m {:.0f}s'.format(
        (end-start) // 60, (end-start) % 60))
        print('Best val Acc: {:4f}'.format(best_acc))
        
        
        model.load_state_dict(best_model_wt)
        return model

In [ ]:
base_model=models.densenet169(pretrained=True)

In [ ]:
for param in base_model.parameters():
    param.requires_grad = False


In [ ]:
in_feat=base_model.classifier.in_features

In [ ]:
model=nn.Sequential(
    base_model,
    nn.Linear(in_feat,1),
)

In [ ]:
model=model.to(device)

In [ ]:
criterion=nn.BCELoss()

In [ ]:
optimizer=optim.Adam(params=model.parameters(),lr=0.01)

In [ ]:
exp_lr_scheduler=lr_scheduler.StepLR(optimizer,step_size=5,gamma=0.3)

In [ ]:
model=train_model(model,criterion,optimizer,exp_lr_scheduler)